# Task 1: Exploratory Data Analysis and Data Preprocessing

**Objective**: Load, explore, filter, clean, and preprocess the CFPB complaints dataset for downstream NLP tasks.

**Dataset**: Consumer Financial Protection Bureau (CFPB) Complaints

**Author**: Data & AI Engineer

**Date**: 2026-01-06

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print("Libraries imported successfully.")

## 2. Load Dataset

Load the full CFPB complaints dataset from the `data/raw/` directory.

In [ ]:
# Define file path
data_path = Path('../data/raw/')
csv_files = list(data_path.glob('*.csv'))

print(f"Found {len(csv_files)} CSV file(s) in data/raw/:")
for f in csv_files:
    print(f"  - {f.name}")

# Load the dataset (assuming the first CSV is the complaints dataset)
if csv_files:
    df_raw = pd.read_csv(csv_files[0], low_memory=False)
    print(f"\nDataset loaded: {csv_files[0].name}")
    print(f"Shape: {df_raw.shape}")
else:
    raise FileNotFoundError("No CSV files found in data/raw/. Please add the CFPB complaints dataset.")

## 3. Initial Data Exploration

Examine the structure, columns, and basic statistics of the dataset.

In [ ]:
# Display first few rows
print("First 5 rows:")
display(df_raw.head())

In [ ]:
# Dataset info
print("Dataset Information:")
df_raw.info()

In [ ]:
# Check for missing values
print("Missing Values:")
missing_df = pd.DataFrame({
    'Column': df_raw.columns,
    'Missing_Count': df_raw.isnull().sum().values,
    'Missing_Percentage': (df_raw.isnull().sum().values / len(df_raw) * 100).round(2)
})
display(missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False))

In [ ]:
# Identify the complaint narrative column
# Common column names: 'Consumer complaint narrative', 'Complaint', 'Consumer Complaint', etc.
narrative_cols = [col for col in df_raw.columns if 'narrative' in col.lower() or 'complaint' in col.lower()]
print(f"Potential narrative columns: {narrative_cols}")

# Identify the product column
product_cols = [col for col in df_raw.columns if 'product' in col.lower()]
print(f"Potential product columns: {product_cols}")

## 4. Exploratory Data Analysis (EDA)

### 4.1 Distribution of Complaints by Product

In [ ]:
# Assuming the product column is named 'Product' (adjust if different)
product_col = 'Product' if 'Product' in df_raw.columns else product_cols[0] if product_cols else None

if product_col:
    # Count complaints by product
    product_counts = df_raw[product_col].value_counts()
    
    print(f"Total unique products: {len(product_counts)}")
    print(f"\nTop 15 Products by Complaint Count:")
    display(product_counts.head(15).to_frame(name='Count'))
    
    # Visualize top 15 products
    plt.figure(figsize=(14, 7))
    product_counts.head(15).plot(kind='barh', color='steelblue')
    plt.xlabel('Number of Complaints', fontsize=12)
    plt.ylabel('Product', fontsize=12)
    plt.title('Top 15 Products by Complaint Count', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
else:
    print("Product column not found. Please verify the dataset structure.")

### 4.2 Count of Complaints With vs Without Consumer Complaint Narratives

In [ ]:
# Assuming the narrative column is named 'Consumer complaint narrative' (adjust if different)
narrative_col = 'Consumer complaint narrative' if 'Consumer complaint narrative' in df_raw.columns else narrative_cols[0] if narrative_cols else None

if narrative_col:
    # Count narratives
    has_narrative = df_raw[narrative_col].notna().sum()
    no_narrative = df_raw[narrative_col].isna().sum()
    total = len(df_raw)
    
    print(f"Total Complaints: {total:,}")
    print(f"With Narrative: {has_narrative:,} ({has_narrative/total*100:.2f}%)")
    print(f"Without Narrative: {no_narrative:,} ({no_narrative/total*100:.2f}%)")
    
    # Visualize
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Bar chart
    categories = ['With Narrative', 'Without Narrative']
    counts = [has_narrative, no_narrative]
    colors = ['#2ecc71', '#e74c3c']
    
    ax1.bar(categories, counts, color=colors, alpha=0.8)
    ax1.set_ylabel('Number of Complaints', fontsize=12)
    ax1.set_title('Complaints With vs Without Narratives', fontsize=14, fontweight='bold')
    ax1.ticklabel_format(style='plain', axis='y')
    
    for i, v in enumerate(counts):
        ax1.text(i, v + total*0.01, f'{v:,}\n({v/total*100:.1f}%)', ha='center', fontsize=10, fontweight='bold')
    
    # Pie chart
    ax2.pie(counts, labels=categories, autopct='%1.1f%%', colors=colors, startangle=90, textprops={'fontsize': 11})
    ax2.set_title('Proportion of Complaints', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print("Narrative column not found. Please verify the dataset structure.")

### 4.3 Word Count Distribution of Complaint Narratives

Analyze the length distribution of complaint narratives to understand text complexity.

In [ ]:
if narrative_col:
    # Filter non-null narratives
    narratives = df_raw[df_raw[narrative_col].notna()][narrative_col]
    
    # Calculate word counts
    word_counts = narratives.apply(lambda x: len(str(x).split()))
    
    # Statistics
    print("Word Count Statistics:")
    print(f"Mean: {word_counts.mean():.2f}")
    print(f"Median: {word_counts.median():.2f}")
    print(f"Std Dev: {word_counts.std():.2f}")
    print(f"Min: {word_counts.min()}")
    print(f"Max: {word_counts.max()}")
    print(f"25th Percentile: {word_counts.quantile(0.25):.2f}")
    print(f"75th Percentile: {word_counts.quantile(0.75):.2f}")
    
    # Visualize distribution
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Histogram
    axes[0].hist(word_counts, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
    axes[0].axvline(word_counts.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {word_counts.mean():.0f}')
    axes[0].axvline(word_counts.median(), color='green', linestyle='--', linewidth=2, label=f'Median: {word_counts.median():.0f}')
    axes[0].set_xlabel('Word Count', fontsize=12)
    axes[0].set_ylabel('Frequency', fontsize=12)
    axes[0].set_title('Distribution of Word Counts in Complaint Narratives', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=10)
    axes[0].grid(axis='y', alpha=0.3)
    
    # Box plot
    axes[1].boxplot(word_counts, vert=True, patch_artist=True, 
                    boxprops=dict(facecolor='lightblue', color='blue'),
                    medianprops=dict(color='red', linewidth=2),
                    whiskerprops=dict(color='blue'),
                    capprops=dict(color='blue'))
    axes[1].set_ylabel('Word Count', fontsize=12)
    axes[1].set_title('Box Plot of Word Counts', fontsize=14, fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Additional analysis: narratives with very few words
    short_narratives = (word_counts < 10).sum()
    print(f"\nNarratives with < 10 words: {short_narratives:,} ({short_narratives/len(word_counts)*100:.2f}%)")
else:
    print("Narrative column not found. Cannot analyze word counts.")

## 5. Data Filtering

Filter the dataset to include ONLY the following products:
- Credit card
- Personal loan
- Savings account
- Money transfer

In [ ]:
if product_col:
    # Define target products (case-insensitive matching)
    target_products = ['Credit card', 'Personal loan', 'Savings account', 'Money transfer']
    
    # Check exact matches first
    print("Checking for exact product matches...")
    exact_matches = df_raw[product_col].isin(target_products)
    print(f"Exact matches found: {exact_matches.sum()}")
    
    # If no exact matches, try case-insensitive partial matching
    if exact_matches.sum() == 0:
        print("\nNo exact matches. Attempting case-insensitive partial matching...")
        print("\nAvailable products in dataset:")
        print(df_raw[product_col].unique()[:20])  # Show first 20 products
        
        # Create a filter using case-insensitive partial matching
        filter_mask = df_raw[product_col].str.lower().str.contains(
            '|'.join([p.lower() for p in target_products]), 
            na=False, 
            regex=True
        )
    else:
        filter_mask = exact_matches
    
    # Apply filter
    df_filtered = df_raw[filter_mask].copy()
    
    print(f"\nOriginal dataset size: {len(df_raw):,}")
    print(f"Filtered dataset size: {len(df_filtered):,}")
    print(f"Reduction: {(1 - len(df_filtered)/len(df_raw))*100:.2f}%")
    
    # Show distribution of filtered products
    print(f"\nProduct distribution after filtering:")
    display(df_filtered[product_col].value_counts().to_frame(name='Count'))
else:
    print("Product column not found. Cannot filter by product.")
    df_filtered = df_raw.copy()

## 6. Remove Rows with Empty or Null Narratives

Remove all rows where the consumer complaint narrative is empty or null.

In [ ]:
if narrative_col:
    print(f"Before removing null narratives: {len(df_filtered):,} rows")
    
    # Remove null and empty narratives
    df_filtered = df_filtered[df_filtered[narrative_col].notna()].copy()
    df_filtered = df_filtered[df_filtered[narrative_col].str.strip() != ''].copy()
    
    print(f"After removing null narratives: {len(df_filtered):,} rows")
    print(f"Rows removed: {len(df_raw[filter_mask]) - len(df_filtered):,}")
else:
    print("Narrative column not found. Cannot remove null narratives.")

## 7. Text Cleaning and Preprocessing

Clean the complaint narratives by:
1. Converting to lowercase
2. Removing special characters and extra whitespace
3. Removing common boilerplate phrases

In [ ]:
def clean_text(text):
    """
    Clean complaint narrative text.
    
    Args:
        text (str): Raw complaint text
    
    Returns:
        str: Cleaned text
    """
    if pd.isna(text):
        return ''
    
    # Convert to string and lowercase
    text = str(text).lower()
    
    # Remove boilerplate phrases (case-insensitive)
    boilerplate_phrases = [
        r'i am writing to file a complaint',
        r'i am filing this complaint',
        r'i would like to file a complaint',
        r'i wish to file a complaint',
        r'this is a complaint regarding',
        r'i am submitting this complaint',
        r'dear sir or madam',
        r'to whom it may concern',
        r'dear cfpb',
    ]
    
    for phrase in boilerplate_phrases:
        text = re.sub(phrase, '', text, flags=re.IGNORECASE)
    
    # Remove special characters (keep letters, numbers, and spaces)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Strip leading/trailing whitespace
    text = text.strip()
    
    return text

# Test the cleaning function
sample_text = "I am writing to file a complaint about my Credit Card!! The company charged me $500.00 (unauthorized)."
print("Original text:")
print(sample_text)
print("\nCleaned text:")
print(clean_text(sample_text))

In [ ]:
if narrative_col:
    # Create a new column for cleaned narratives
    print("Cleaning complaint narratives...")
    df_filtered['cleaned_narrative'] = df_filtered[narrative_col].apply(clean_text)
    
    # Remove rows where cleaned narrative is empty
    initial_count = len(df_filtered)
    df_filtered = df_filtered[df_filtered['cleaned_narrative'].str.strip() != ''].copy()
    final_count = len(df_filtered)
    
    print(f"Rows removed after cleaning (empty after processing): {initial_count - final_count:,}")
    print(f"Final dataset size: {final_count:,}")
    
    # Show examples
    print("\n" + "="*80)
    print("SAMPLE BEFORE AND AFTER CLEANING")
    print("="*80)
    
    for i in range(min(3, len(df_filtered))):
        idx = df_filtered.index[i]
        print(f"\nExample {i+1}:")
        print("-" * 80)
        print("ORIGINAL:")
        print(df_filtered.loc[idx, narrative_col][:300] + "..." if len(df_filtered.loc[idx, narrative_col]) > 300 else df_filtered.loc[idx, narrative_col])
        print("\nCLEANED:")
        print(df_filtered.loc[idx, 'cleaned_narrative'][:300] + "..." if len(df_filtered.loc[idx, 'cleaned_narrative']) > 300 else df_filtered.loc[idx, 'cleaned_narrative'])
else:
    print("Narrative column not found. Cannot clean narratives.")

## 8. Final Dataset Statistics

Summary statistics of the cleaned and filtered dataset.

In [ ]:
print("FINAL DATASET SUMMARY")
print("="*80)
print(f"Total rows: {len(df_filtered):,}")
print(f"Total columns: {len(df_filtered.columns)}")
print(f"\nColumns: {list(df_filtered.columns)}")

if product_col:
    print(f"\nProduct Distribution:")
    display(df_filtered[product_col].value_counts().to_frame(name='Count'))

if 'cleaned_narrative' in df_filtered.columns:
    # Word count statistics for cleaned narratives
    cleaned_word_counts = df_filtered['cleaned_narrative'].apply(lambda x: len(str(x).split()))
    
    print(f"\nCleaned Narrative Word Count Statistics:")
    print(f"Mean: {cleaned_word_counts.mean():.2f}")
    print(f"Median: {cleaned_word_counts.median():.2f}")
    print(f"Std Dev: {cleaned_word_counts.std():.2f}")
    print(f"Min: {cleaned_word_counts.min()}")
    print(f"Max: {cleaned_word_counts.max()}")

print(f"\nMemory usage: {df_filtered.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 9. Save Cleaned Dataset

Save the cleaned and filtered dataset to `data/filtered_complaints.csv`.

In [ ]:
# Define output path
output_path = Path('../data/filtered_complaints.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)

# Save to CSV
df_filtered.to_csv(output_path, index=False)

print(f"Cleaned dataset saved to: {output_path}")
print(f"File size: {output_path.stat().st_size / 1024**2:.2f} MB")
print(f"\nDataset shape: {df_filtered.shape}")
print(f"Columns saved: {list(df_filtered.columns)}")

## 10. Verification

Verify the saved dataset can be loaded correctly.

In [ ]:
# Load the saved dataset
df_verify = pd.read_csv(output_path)

print("Verification:")
print(f"Loaded shape: {df_verify.shape}")
print(f"Original shape: {df_filtered.shape}")
print(f"Match: {df_verify.shape == df_filtered.shape}")

# Display first few rows
print("\nFirst 3 rows of saved dataset:")
display(df_verify.head(3))

print("\n" + "="*80)
print("TASK 1 COMPLETED SUCCESSFULLY")
print("="*80)